# December 11, 2022
https://adventofcode.com/2022/day/11

In [1]:
fn = "../data/2022/11.txt"

desc = []
puz = []
with open(fn, "r") as file:
    while True:
        
        line = file.readline()

        if not line:
            puz.append(desc)
            break
        
        if line == "\n":
            puz.append(desc)
            desc = []
        else:
            desc.append(line.strip())


In [2]:
test_desc = [
f"""Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3""",
f"""Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0""",
f"""Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3"""
,
f"""Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
"""]


test = [ [ x.strip() for x in test_monkey.split("\n") ] for test_monkey in test_desc ]

In [31]:
class Monkey:
    # class member needed for Part 2
    # I should really inherit this class in case I want multiple sets of monkeys... haha
    ring_size = 3 # must be set later
    monkey_style = 1
    @classmethod
    def set_ring_size( cls, x ):
        cls.ring_size = x

    def __init__( self, desc ):
        # Parse items
        item_list = desc[1][ len("Starting items: "):]
        self.items = [int(x) for x in item_list.split(", ")]

        # Parse inspection description
        insp_desc = desc[2][ len("Operation: new = "):]
        insp_desc = insp_desc.split(" ")

        if insp_desc[1] == "*":
            if insp_desc[2] == "old":
                self.inspect_fun = lambda x: x*x
            else:
                self.inspect_fun = lambda x: x*int(insp_desc[2])
        elif insp_desc[1] == "+":
            if insp_desc[2] == "old":
                self.inspect_fun = lambda x: x+x
            else:
                self.inspect_fun = lambda x: x+int(insp_desc[2])
        else:
            raise Exception("Did not understand inspection description!")

        # Parse test description
        test_desc = desc[3][ len("Test: "): ]
        test_desc = test_desc.split()

        if test_desc[0] == "divisible" and test_desc[1] == "by":
            self.test_fun = lambda x: x % int(test_desc[2]) == 0
            self.divisor = int(test_desc[2]) # part 2
        else:
            raise Exception("Did not understand test description!")

        # Parse partner monkeys
        self.monkey1 = int(desc[4][len("If true: throw to monkey "):])
        self.monkey0 = int(desc[5][len("If false: throw to monkey "):])

        # Start counter:
        self.activity = 0

    # inspect the next item and then toss it
    def inspect( self ):
        if len(self.items) == 0:
            return None
        
        self.activity += 1
        
        # inspect the item
        x = self.items[0]
        if self.monkey_style == 1:
            x = int( self.inspect_fun( x ) / 3 ) # Part 1
        elif self.monkey_style == 2:
            x = int( self.inspect_fun(x) % self.ring_size)
        else:
            raise Exception("Illegal monkey style!")

        flag = self.test_fun( x )

        # toss it
        self.items = self.items[1:]
        if flag:
            return self.monkey1, x
        else:
            return self.monkey0, x

    def catch( self, item ):
        self.items.append(item)

    def __str__( self ):
        return f"""Activity Level: {self.activity}\nItems: {", ".join([str(x) for x in self.items])}"""

In [32]:
# function for Part 2
def lcm( monkeys ):
  # Least Common Monkey 
  # This isn't the "least" common multiple, but we just need any multiple for this
  mult = 1
  for mon in monkeys:
      mult *= mon.divisor
  return mult

def reset_test(part2=False):
  global test
  global test_monkeys
  test_monkeys = [Monkey(x) for x in test]

  if part2:
    mult = lcm(test_monkeys)
    Monkey.monkey_style = 2
  else:
    mult = 3
    Monkey.monkey_style = 1
  Monkey.set_ring_size( mult )

def reset_puz(part2=False):
  global puz
  global puz_monkeys
  puz_monkeys = [Monkey(x) for x in puz]

  if part2:
    mult = lcm(puz_monkeys)
  else:
    mult = 3
  Monkey.set_ring_size( mult )
  


In [33]:
test_monkey = Monkey(test[0])
Monkey.set_ring_size(3)
Monkey.monkey_style = 1
print(test_monkey.items)

while True:
    out = test_monkey.inspect()
    if not out:
        break
    target, item = out
    print( f"Throw item {item} to monkey {target}")
    print(test_monkey.items)

assert len(test_monkey.items) == 0

[79, 98]
Throw item 500 to monkey 3
[98]
Throw item 620 to monkey 3
[]


In [34]:
def inspect_all_goods( monkeys, mon ):
    while True:
        out = mon.inspect()
        if not out:
            break
        target, item = out
        #print(target, item)
        monkeys[target].catch(item)

    
def play_a_round( monkeys ):
    for i,mon in enumerate(monkeys):
        #print(i)
        inspect_all_goods( monkeys, mon )

def print_monkeys( monkeys ):
    for i,mon in enumerate(monkeys):
        print(f"Monkey {i}:")
        print(mon)
        #print("\n")

### Part 1

In [35]:
reset_test()

print_monkeys(test_monkeys)
for i in range(20):
    print("-----\n")
    play_a_round(test_monkeys)
    print_monkeys(test_monkeys)

Monkey 0:
Activity Level: 0
Items: 79, 98
Monkey 1:
Activity Level: 0
Items: 54, 65, 75, 74
Monkey 2:
Activity Level: 0
Items: 79, 60, 97
Monkey 3:
Activity Level: 0
Items: 74
-----

Monkey 0:
Activity Level: 2
Items: 20, 23, 27, 26
Monkey 1:
Activity Level: 4
Items: 2080, 25, 167, 207, 401, 1046
Monkey 2:
Activity Level: 3
Items: 
Monkey 3:
Activity Level: 5
Items: 
-----

Monkey 0:
Activity Level: 6
Items: 695, 10, 71, 135, 350
Monkey 1:
Activity Level: 10
Items: 43, 49, 58, 55, 362
Monkey 2:
Activity Level: 4
Items: 
Monkey 3:
Activity Level: 10
Items: 
-----

Monkey 0:
Activity Level: 11
Items: 16, 18, 21, 20, 122
Monkey 1:
Activity Level: 15
Items: 1468, 22, 150, 286, 739
Monkey 2:
Activity Level: 4
Items: 
Monkey 3:
Activity Level: 15
Items: 
-----

Monkey 0:
Activity Level: 16
Items: 491, 9, 52, 97, 248, 34
Monkey 1:
Activity Level: 20
Items: 39, 45, 43, 258
Monkey 2:
Activity Level: 4
Items: 
Monkey 3:
Activity Level: 20
Items: 
-----

Monkey 0:
Activity Level: 22
Items: 15, 17

In [36]:
reset_puz()

#print_monkeys(puz_monkeys)
for i in range(20):
    #print("-----\n")
    play_a_round(puz_monkeys)
    #print_monkeys(puz_monkeys)

activity = [ mon.activity for mon in puz_monkeys ]
activity.sort()
print(activity)
print(activity[-2] * activity[-1])

[9, 34, 70, 167, 295, 311, 312, 320]
99840


### Part 2

In [38]:
reset_test(part2=True)
print_monkeys(test_monkeys)

for i in range(10000):
    if i in [1,20]:
        print("After round", i)
        print_monkeys(test_monkeys)
    elif i % 1000 == 0:
        print("After round", i)
        print([x.activity for x in test_monkeys])
    play_a_round(test_monkeys)


print([x.activity for x in test_monkeys])

Monkey 0:
Activity Level: 0
Items: 79, 98
Monkey 1:
Activity Level: 0
Items: 54, 65, 75, 74
Monkey 2:
Activity Level: 0
Items: 79, 60, 97
Monkey 3:
Activity Level: 0
Items: 74
After round 0
[0, 0, 0, 0]
After round 1
Monkey 0:
Activity Level: 2
Items: 60, 71, 81, 80
Monkey 1:
Activity Level: 4
Items: 77, 1504, 1865, 6244, 3603, 9412
Monkey 2:
Activity Level: 3
Items: 
Monkey 3:
Activity Level: 6
Items: 
After round 20
Monkey 0:
Activity Level: 99
Items: 7723, 61208, 82089, 95446, 84350
Monkey 1:
Activity Level: 97
Items: 84591, 55901, 10567, 20200, 60575
Monkey 2:
Activity Level: 8
Items: 
Monkey 3:
Activity Level: 103
Items: 
After round 1000
[5204, 4792, 199, 5192]
After round 2000
[10419, 9577, 392, 10391]
After round 3000
[15638, 14358, 587, 15593]
After round 4000
[20858, 19138, 780, 20797]
After round 5000
[26075, 23921, 974, 26000]
After round 6000
[31294, 28702, 1165, 31204]
After round 7000
[36508, 33488, 1360, 36400]
After round 8000
[41728, 38268, 1553, 41606]
After round 90

In [41]:
reset_puz(part2=True)
print(Monkey.ring_size)
print(Monkey.monkey_style)

for i in range(10000):
    if i % 1000 == 0:
        print("After round", i)
        print([x.activity for x in puz_monkeys])
    play_a_round(puz_monkeys)


activity = [x.activity for x in puz_monkeys]
activity.sort()
activity[-1] * activity[-2]

9699690
2
After round 0
[0, 0, 0, 0, 0, 0, 0, 0]
After round 1000
[14399, 1929, 2654, 12646, 5440, 12847, 6091, 14203]
After round 2000
[28912, 4059, 5350, 25198, 10283, 25647, 12082, 28473]
After round 3000
[43427, 6184, 8040, 37753, 15122, 38445, 18079, 42742]
After round 4000
[57937, 8312, 10730, 50305, 19935, 51241, 24083, 57010]
After round 5000
[72452, 10435, 13416, 62868, 24770, 64045, 30078, 71279]
After round 6000
[86964, 12558, 16106, 75420, 29592, 76849, 36072, 85547]
After round 7000
[101474, 14686, 18797, 87977, 34414, 89650, 42070, 99811]
After round 8000
[115992, 16812, 21486, 100533, 39243, 102449, 48064, 114086]
After round 9000
[130507, 18938, 24185, 113091, 44093, 115246, 54053, 128358]


20683044837